In [29]:
import pandas as pd
import geopandas as gpd
from matplotlib.colors import Normalize

In [3]:
def csv_to_df(file_path:str):
    """
    Takes in Relative Path for CSV file
    Returns pandas dataframe
    """
    return pd.read_csv(file_path)


In [14]:
df = csv_to_df("../data/2021-SxS-Data-and-Data-Info/partial_cleaned_2021.csv")

In [24]:
def gen_big_3_df(df):
    feature_df = df[['state','math_conf_lvl','fin_sat_lvl','fin_conf_lvl']]

    # Define the columns and values to replace
    columns_dict = {
        'math_conf_lvl': [99, 98],
        'fin_sat_lvl': [99, 98],
        'fin_conf_lvl': [99, 98],
   
    }
    # Loop over the dictionary to replace the values
    for column, values in columns_dict.items():
        for value in values:
            feature_df.loc[feature_df[column] == value, column] = None
    feature_df = feature_df.dropna()
    return feature_df

In [26]:
big_3_df = gen_big_3_df(df)

In [27]:
def gen_geo_df(mini_df):
    state_math_avg =df[['state','math_conf_lvl']].groupby('state').mean().reset_index()
    state_math_avg=state_math_avg.sort_values(by='state')

    state_sat_avg =df[['state','fin_sat_lvl']].groupby('state').mean().reset_index()
    state_sat_avg=state_sat_avg.sort_values(by='state')

    fin_conf_avg = df[['state','fin_conf_lvl']].groupby('state').mean().reset_index()
    fin_conf_avg = fin_conf_avg.sort_values(by='state')
    # Load file
    url = "https://raw.githubusercontent.com/holtzy/The-Python-Graph-Gallery/master/static/data/us_states_hexgrid.geojson.json"
    geoData = gpd.read_file(url)
    # Add a new column to the geo dataframe that will be used for joining:
    geoData['state'] = geoData['google_name'].str.replace('(United States)','')
    # Sort by column: 'state' (ascending)
    geoData = geoData.sort_values(['state']).reset_index()
    # Drop column: 'bees'
    geoData = geoData.drop(columns=['bees', 'index'])
    #Create Columns from subsets
    geoData['fin_sat_avg'] = state_sat_avg['fin_sat_lvl']
    geoData['math_conf_avg'] = state_math_avg['math_conf_lvl']
    geoData['fin_conf_avg'] = fin_conf_avg['fin_conf_lvl']
    return geoData


In [ ]:
gen_geo_df(big_3_df)

In [12]:
def gen_hexbin_map(geo_df):
    """
    Generated hexbin map based on user selection
    """
    # Initialize the figure
fig, ax = plt.subplots(1, figsize=(13, 13))


# Draw a map with matplotlib 
geoData.plot(
    ax=ax, 
    column='fin_sat_avg',
    cmap="BuPu",
    edgecolor='black', 
    linewidth=.5,
    legend=False
)
# add a "centroid" column with the centroid position of each state
geoData['centroid'] = geoData['geometry'].apply(lambda x: x.centroid)

# for each state, annotate with the state name located at the centroid coordinates 
for idx, row in geoData.iterrows():
    ax.annotate(
        text=row['iso3166_2'], 
        xy=row['centroid'].coords[0], 
        horizontalalignment='center', 
        va='center',
        color= 'black'
    )
# Remove axis
ax.axis('off')

plt.title('Average Financial Satisfaction' , size= 20)

# Add a color bar
norm = Normalize(vmin=geoData['fin_sat_avg'].min(), vmax=geoData['fin_sat_avg'].max())
sm = plt.cm.ScalarMappable(cmap='BuPu', norm=norm)
sm.set_array([geoData['fin_sat_avg']]) 
fig.colorbar(sm, orientation="horizontal",  aspect=50,fraction=0.005, pad=0, ax = ax )
    return f'You picked {option}'

In [35]:
feature = 'Finacial Satisfaction'
options = {
    'Finacial Satisfaction': 'fin_sat_lvl', 
    'Financial Confidence': 'fin_conf_lvl', 
    'Math Confidence': 'math_conf_lvl'
    }

selection = [options[feature]]

In [36]:
selection

['fin_sat_lvl']